# Домашнее задание № 4. Языковые модели

## Задание 1 (8 баллов).

В семинаре для генерации мы использовали предположение маркова и считали, что слово зависит только от 1 предыдущего слова. Но ничто нам не мешает попробовать увеличить размер окна и учитывать два или даже три прошлых слова. Для них мы еще сможем собрать достаточно статистик и, логично предположить, что качество сгенерированного текста должно вырасти.

Попробуйте сделать языковую модель, которая будет учитывать два предыдущих слова при генерации текста.
Сгенерируйте несколько текстов (3-5) и расчитайте перплексию получившейся модели. 
Можно использовать данные из семинара или любые другие (сопоставимые или большие по объему). Перплексию рассчитывайте на 10-50 отложенных предложениях (они не должны использоваться при сборе статистик).


Подсказки:  
    - нужно будет добавить еще один тэг <start>  
    - еще одна матрица не нужна, можно по строкам хронить биграмы, а по колонкам униграммы  
    - тексты должны быть очень похожи на нормальные (если у вас получается рандомная каша, вы что-то делаете не так). 

In [22]:
from string import punctuation
from razdel import sentenize
from razdel import tokenize as razdel_tokenize
import numpy as np
from IPython.display import Image
from IPython.core.display import HTML 

In [23]:
#dvach = open('2ch_corpus.txt', encoding="utf8").read()
news = open('lenta.txt', encoding="utf8").read()

In [24]:
def normalize(text):
    normalized_text = [word.text.strip(punctuation) for word \
                                                            in razdel_tokenize(text)]
    normalized_text = [word.lower() for word in normalized_text if word and len(word) < 20 ]
    return normalized_text

#norm_dvach = normalize(dvach)
norm_news = normalize(news)

In [25]:
from collections import Counter
#vocab_dvach = Counter(norm_dvach)
vocab_news = Counter(norm_news)

In [26]:
#probas_dvach = Counter({word:c/len(norm_dvach) for word, c in vocab_dvach.items()})
probas_news = Counter({word:c/len(norm_news) for word, c in vocab_news.items()})

In [27]:
from nltk.tokenize import sent_tokenize
def ngrammer(tokens, n=2):
    ngrams = []
    for i in range(0,len(tokens)-n+1):
        ngrams.append(' '.join(tokens[i:i+n]))
    return ngrams

In [28]:
sentences_news = [['<start>']+['<start>'] + normalize(text) + ['<end>'] for text in sent_tokenize(news)]
sentences_news_test = sentences_news[:50]
sentences_news = sentences_news[50:]
#sentences_news = [['<start>'] + normalize(text) + ['<end>'] for text in sent_tokenize(news)]

In [29]:
unigrams_news = Counter()
bigrams_news = Counter()
trigrams_news = Counter()

for sentence in sentences_news:
    unigrams_news.update(sentence)
    bigrams_news.update(ngrammer(sentence))
    trigrams_news.update(ngrammer(sentence, 3))

In [30]:
from scipy.sparse import csr_matrix, csc_matrix, lil_matrix

matrix_news = lil_matrix((len(bigrams_news), len(unigrams_news)))

id2word_news = list(unigrams_news)
word2id_news = {word:i for i, word in enumerate(id2word_news)}

id2bigram_news = list(bigrams_news)
bigram2id_news = {word:i for i, word in enumerate(id2bigram_news)}

for ngram in trigrams_news:
    word1, word2, word3 = ngram.split()
    bigram = word1 + ' '+ word2 

    matrix_news[bigram2id_news[bigram], word2id_news[word3]] =  (trigrams_news[ngram]/bigrams_news[bigram])

matrix_news = csr_matrix(matrix_news)

In [31]:
def generate(matrix, id2bigram, id2word, bigram2id, n=100, start='<start> <start>'):
    text = []
    current_idx = bigram2id[start]
    for i in range(n):
        p = matrix[current_idx].toarray()[0]
        chosen = np.random.choice(list(range(matrix.shape[1])), p = p)
        text.append(id2word[chosen])
        if id2word[chosen] == '<end>':
            current_idx = bigram2id[start]
        else:
            part = id2bigram[current_idx] + ' ' + id2word[chosen]
            part = ' '.join(part.split()[1:])
            current_idx = bigram2id[part]
    
    return ' '.join(text)

In [32]:
print(generate(matrix_news, id2bigram_news, id2word_news, bigram2id_news, start='<start> <start>').replace('<end>', '\n'))

500 миллионов фунтов так что на нескольких серверах русской паутины на входных страницах поисковых машин оставалась одной из секций тепловоза товарного поезда взорвалась подложенная мина об этом риа новости 
 как отметил суд дальнейшее содержание шутова под стражей было бы обратиться в международный суд стокгольма принял решение пригласить папу римского иоанна павла ii 
 предполагается что в понедельник 25 октября поздно вечером завершился закрытый пленум московского городского суда в качестве чернорабочего 
 этот документ не более 30 военнослужащих федеральных сил предотвратили диверсию на железнодорожных вокзалах в армавире и пятигорске в апреле этого года саммита группы восьми совета европы указал на


In [33]:
def perplexity(probas):
    p = np.exp(np.sum(probas))
    N = len(probas)
    
    return p**(-1/N) 

In [35]:
def count_perplexity(sentences):
    all_perplexity = []
    for sent in sentences_news_test:
        probs = []
        for ngram in ngrammer(sent, 3):
            word1, word2, word3 = ngram.split()
            bigram = word1 + ' '+word2
            if ngram in trigrams_news and bigram in bigrams_news:
                probs.append(np.log(trigrams_news[ngram]/bigrams_news[bigram]))
            else:
                probs.append(np.log(0.00001))
                if perplexity(probs)!= np.inf: 
                    all_perplexity.append(perplexity(probs))
                
    return np.mean(all_perplexity)

In [36]:
count_perplexity(sentences_news_test)

<ipython-input-33-29b2627a57da>:5: RuntimeWarning: divide by zero encountered in double_scalars
  return p**(-1/N)


64102.96600416621

## Задание № 2* (2 балла). 

Прочитайте главу про языковое моделирование в книге Журафски и Мартина - https://web.stanford.edu/~jurafsky/slp3/3.pdf

Развернуто (в пределах 1000 знаков) ответьте на вопросы (по-русски):

1. Что можно делать с проблемой несловарных слов? В семинаре мы просто использовали какое-то маленькое значение вероятности, а какие есть другие способы?

2. Что такое сглаживание (smoothing)?

1. Часто для языковых моделей используются системы "закрытых" словарей, то есть словарей с ограниченным количеством слов. Им противопоставляются системы "открытых" словарей, где мы можем моделировать потенциально незнакомые слова в тестовом наборе данных через добавление псевдо-слов. Есть два популярных способа обучить вероятности появления неизвестного слова.
а) Первый способ - все же выбрать закрытый словарь заранее:
- выбрать фиксированный словарь
- на шаге нормализации в тестовом датасете конвертировать любое слово НЕ в этом датасете в <UNK> - "unknown word token".
- посчитать вероятость <"UNK">
б) Второй способ - если у нас нет готового словаря заранее, состоит в том, чтобы создать словарь путем замены слов в обучающем датасете на <"UNK"> в зависимости от их частотности. То есть если какие-то слова встречаются супер редко, их можно заменить на UNK. 

    2. Бывают ситуации, когда слова в тестовом датасете встречаются в контексте, в котором не встречались в обучающем. В таком случае, модель будет присваивать им нулевую вероятность встретиться в этом контексте. Чтобы этого избежать, берется вероятность у других событий и распределяется между событиями, которые не встречались. Это и называется сглаживанием.
    Способов сглаживания много, но самый простой заключается в том, чтобы самостоятельно увеличить количество н-грамм на 1 перед тем, как считать вероятность. Так что теперь если н-грамма встречалась один раз - будем считать, что 2, если не встречалась вообще, будем считать, что встретилась один раз. Это называется сглаживанием Лапласа.
   